# Interactive apps with Panel

Solution to the exercises from `panel_spectrum.ipynb`

`Author: Nikolai Kapralov`

In [ ]:
import mne
import matplotlib.pyplot as plt
import panel as pn

from mne_bids import print_dir_tree
from pathlib import Path

pn.extension(sizing_mode='stretch_width',
             theme='default')

### Interactivity and Widgets

Source: https://panel.holoviz.org/how_to/interactivity/index.html

For the minimal example, we can use a function that creates a string by repeating a provided symbol given number of times. We will bind its arguments to UI widgets and make the function interactive!

In [ ]:
def string_creator(symbol, number):
    return symbol * number

In [ ]:
string_creator('🌊', 5)

Now we need to create two widgets corresponding to the arguments of the function. Overall, there are lots of different widgets ([listed here](https://panel.holoviz.org/reference/index.html#widgets)), and often there are multiple options to choose from depending on the context. Here and in the next examples, we will use select/drop-down for choosing between several _textual_ options and slider for setting the _numeric_ values.

The core parameter of many widgets is `value`, which contains the current value and also allows setting the initial one. Other parameters may include display name (`name`) or widget-specific input constraints (e.g., `start` and `end` for the slider range or `options` to select from).

Below are the widgets that are helpful for this minimal example:

In [ ]:
slider = pn.widgets.IntSlider(name='Length', value=5, start=1, end=10)
slider

Change the position of slider and re-run the cell below to see the updated value:

In [ ]:
slider.value

In [ ]:
select = pn.widgets.Select(name='Symbol', options=['🌊', '🐘'])
select

In [ ]:
select.value

The next essential step is to combine (bind) together the input values from the widgets with the function that generates the output. The function `bind` serve for this purpose.

It accepts the function as the first argument and several objects that will be provided to this function (the number of provided objects should match the number of arguments). In case of widgets `widget.value` will be automatically passed to the function as in the example below:

```
def my_fun(arg1, arg2):
    pass

pn.bind(my_fun, arg1=obj1, arg2=widget2)
# obj1 will be passed as-is
# widget2.value will be passed as arg2
```

In [ ]:
result = pn.bind(string_creator, select, slider)

Now we can arrange the input (slider, drop-down) and output (text) elements in a layout using row and column wrapper:

In [ ]:
toy_panel = pn.Column('## Minimal Example', 
    pn.Row(slider, select), 
    pn.Row('Updated on every detected change:', result), 
)

The last step is to serve the panel. Here comes interactivity:

In [ ]:
toy_panel.servable()

As you may have noticed, the output is updated all the time - even if you have not released the mouse button yet. While it works fine in this toy example, running more complex computations may take more time and make the app freeze. To avoid this problem, one option is to use `widget.param.value_throttled`, which get updated only when the user releases the mouse button.

In [ ]:
result_throttled = pn.bind(string_creator, select, 
                           slider.param.value_throttled)

pn.Column('## Minimal Example', 
          pn.Row(slider, select), 
          pn.Row('Updated only when the mouse button is released:', result_throttled), 
).servable()  # notice the chain call

The throttled option will come in handy for the coming MNE-based analysis.

**Exercise:**

1. Add one or more widgets to the app
2. Update the `string_creator` function to use the value from the new widget somehow

Here is the list of widgets available in Panel:
https://panel.holoviz.org/reference/index.html#widgets

Some ideas (feel free to follow your own ones):

- TextInput: add custom prefix in the beginning of the string
- IntRangeSlider: generate a string of random length in the specified range

**Solution**:

Function:

In [ ]:
import random

def string_creator_extended(symbol, length_range, prefix):
    result = symbol * random.randint(length_range[0], length_range[1]) # or *length_range
    if prefix:
        return f'{prefix}: {result}'
    
    return result

Widgets:

In [ ]:
select = pn.widgets.Select(name='Symbol', options=['🌊', '🐘'])
length_range = pn.widgets.IntRangeSlider(name='Length limits', 
                                         value=(2, 7), start=1, end=15, step=1)
prefix = pn.widgets.TextInput(name='Prefix')

Panel:

In [ ]:
result_extended = pn.bind(string_creator_extended, select, 
                          length_range.param.value_throttled, prefix)

pn.Column('## Minimal Example', 
          pn.Row(prefix, select, length_range), 
          pn.Row('Updated only when the mouse button is released:', result_extended), 
).servable()  # notice the chain call

### Dataset

Now let's move on to the MNE-based analysis. First, we need to set up the location of the dataset in the variable `path_to_data` below.

In [ ]:
path_to_data = None
if path_to_data is None:
    raise RuntimeError("Please specify the full path to the dataset in the `path_to_data` variable")

If the correct path is provided, the command below should print the list of files and folders contained in the dataset (first level only, e.g., `sub-XX` folders).

In [ ]:
print_dir_tree(path_to_data, max_depth=1)

### Loading and Exploring the Data

Now let's have a quick look at the data before proceeding with the interactive part.

In [ ]:
path_to_file = Path(path_to_data) / 'sub-01' / 'sub-01-raw.fif'
raw = mne.io.read_raw_fif(path_to_file)

In [ ]:
raw.info

In [ ]:
%matplotlib qt
raw.plot();

The dataset contains MEG, EEG, and ECG recordings, which we will use to calculate the spectra. Let's get the indices of these channels.

In [ ]:
meeg_idx = mne.pick_types(raw.info, meg=True, eeg=True)
meeg_ch_names = mne.pick_info(raw.info, meeg_idx)['ch_names']

### Computing the power spectral density (PSD) using Welch and multitaper methods

Computing the PSD in MNE-Python is as easy as this:

In [ ]:
%matplotlib qt
_ = raw.compute_psd().plot();

However, the computation of the spectra is actually based on several parameters that can affect the estimation, and default values are used when no arguments are provided. Let's create an app the would allow us to explore the influence of these parameters on the estimated spectra in an interactive way.

Here is the list of parameters we would like to explore:

 * `channel` - select the channel to analyze (`pn.widgets.Select`)
 * `fmin`, `fmax` - frequency range to use for estimation of the spectra (`pn.widgets.RangeSlider`)
 * `tmin`, `tmax` - time range to use for estimation of the spectra (`pn.widgets.RangeSlider`)

In addition, MNE provides two methods for calculation of the spectra - Welch and multitaper - with a set of method-specific parameters, which we would also add (as an exercise).

Let's define the input widgets:

In [ ]:
# Common parameters
channel_select = pn.widgets.Select(name='Channel', options=meeg_ch_names)
time_slider = pn.widgets.RangeSlider(name='Time Range (s)', 
                                     start=0, end=len(raw)/raw.info['sfreq'], step=10)
freq_slider = pn.widgets.RangeSlider(name='Frequency Range (Hz)', value=(0, 100),
                                     start=0, end=raw.info['sfreq'] // 2, step=5)

# Welch parameters (see the exercise at the end of the notebook)
n_fft_slider = pn.widgets.IntSlider(name='n_fft', value=int(raw.info['sfreq']),
                                    start=100, end=5000, step=100)
n_overlap_slider = pn.widgets.IntSlider(name='n_overlap', value=0,
                                        start=0, end=5000, step=100)
window_select = pn.widgets.Select(name='Window',
                                  options=['hamming', 'triang', 'hann'])

# Multitaper parameters (see the exercise at the end of the notebook)
normalization_select = pn.widgets.Select(name='Normalization',
                                         options=['full', 'length'])
bandwidth_slider = pn.widgets.FloatSlider(name='Half-Bandwidth', value=4,
                                          start=1, end=16, step=1)
low_bias_checkbox = pn.widgets.Checkbox(name='Low bias',
                                        value=True)

Two functions below act as wrappers around `raw.compute_psd.plot` with different parameters and return Matplotlib figures (spectra plots). These figures will be included in the panel side-by-side to compare Welch (`plot_welch_with_params`) and multitaper (`plot_taper_with_params`) methods.

In [ ]:
def plot_welch_with_params(raw, picks, f_range, t_range, 
                           n_fft, n_overlap, window):
    fmin, fmax = f_range
    tmin, tmax = t_range

    fig, ax = plt.subplots()
    fig = raw.compute_psd(picks=picks, method='welch',
                          fmin=fmin, fmax=fmax, 
                          tmin=tmin, tmax=tmax,
                          n_fft=n_fft,
                          n_overlap=n_overlap,
                          window=window)\
             .plot(axes=ax, show=False);
    ax.set_title('Welch')

    plt.close(fig)
    return fig

In [ ]:
def plot_taper_with_params(raw, picks, f_range, t_range, 
                           normalization, half_bandwidth, low_bias):
    fmin, fmax = f_range
    tmin, tmax = t_range

    bandwidth = half_bandwidth * 2 / (tmax - tmin)

    fig, ax = plt.subplots()
    fig = raw.compute_psd(picks=picks, method='multitaper',
                          fmin=fmin, fmax=fmax, 
                          tmin=tmin, tmax=tmax,
                          normalization=normalization,
                          bandwidth=bandwidth,
                          low_bias=low_bias)\
             .plot(axes=ax, show=False);
    ax.set_title('Multitaper')

    plt.close(fig)
    return fig

Now let's bind input and output, arrange all the elements in a panel, and serve it:

In [ ]:
%matplotlib inline
mne.set_log_level('warning')

p1 = pn.bind(plot_welch_with_params, 
             raw=raw, 
             picks=channel_select, 
             f_range=freq_slider.param.value_throttled, 
             t_range=time_slider.param.value_throttled,
             n_fft=n_fft_slider.param.value_throttled,
             n_overlap=n_overlap_slider.param.value_throttled,
             window=window_select);
p2 = pn.bind(plot_taper_with_params, 
             raw=raw, 
             picks=channel_select, 
             f_range=freq_slider.param.value_throttled, 
             t_range=time_slider.param.value_throttled,
             normalization=normalization_select,
             half_bandwidth=bandwidth_slider.param.value_throttled,
             low_bias=low_bias_checkbox);

pn.Column(
    pn.Row('### PSD: Welch vs Multitaper'),
    pn.Row('**Common:**', channel_select, time_slider, freq_slider),
    pn.Row('**Welch:**', n_fft_slider, n_overlap_slider, window_select),
    pn.Row('**Multitaper:**', normalization_select, bandwidth_slider, low_bias_checkbox),
    pn.Row(pn.pane.Matplotlib(p1, tight=True),
           pn.pane.Matplotlib(p2, tight=True))
).servable()

**Exercise:**

1. Add Welch-specific parameters to the app:
 * `n_fft` - the size of data segments (in samples) used for FFT
 * `n_overlap` - the overlap of data segments (in samples)
 * `window` - the type of window applied to the data

2. Add multitaper-specific parameters to the app:
 * `normalization` - how the spectra values are normalized
 * `bandwidth` - average neighboring `bandwidth` frequency bins
 * `low_bias` - whether estimates (tapers) with high bias should be dropped